**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act return the action to do when the agent is in the state s. We can choose two behaviors. If we choose train == False the agent will just choose a learned action to do in this state. If we choose train == True, the agent has a probability $p= 1- \epsilon$ to do the learned action in this state, and a probability $1-p = \epsilon$ to do a random action. This possibility with the $\epsilon$ is essential if we want that the agent doesn't just choose an actual learned action but try to explore sometimes other actions which could have better rewards. It's is essential to continue to learn and improve the decisions of the agent.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 #correction of self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=3 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

- the array **position** contains the information of the position of the rat on the board and the limits of the game. It's a matrix with 0 in the cells where the rat is not, with 1 in the unique cell where the rat is, and -1 if the cells are out of the island (the rat can not go here).
- the array **board** contains the information of the positions of the cheese and the poisonous cells. It's a matrix with +0.5 in the cells where there is cheese, -1 in the cells where there is poison, 0 in the cells with nothing.

The parameter temperature allows to influence the number of cheeses and poisons on the board. Board is first fill with cheese then with poison in the remaining empty cells. The temperature is the probability for a cell to contain a cheese. And it's also the probability for  a cell to contain a poison (if we do not put cheese in before). **The number of cheese and poison on the board increase with the temperature.**

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # choose a random action in [0, 1, 2, 3]
        a = np.random.randint(0, 4)
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state = env.reset()
        game_over = False
        
        win = 0
        lose = 0
        
        while not game_over:
            
            # the agent chooses an action
            action = agent.act(state)
            # act on the environment
            prev_state = state
            state, reward, game_over = env.act(action)
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
            
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 8.0/11.0. Average score (-3.0)
Win/lose count 11.0/14.0. Average score (-3.0)
Win/lose count 7.0/10.0. Average score (-3.0)
Final score: -3.0


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**1)** We have:

\begin{align}
Q^\pi(s,a) &= E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
           &= E_{p^{\pi}}[r(s_{0},a_{0}) + \sum_{1\leq t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
           &= E_{p^{\pi}}[r(s_{0},a_{0})|s_{0}=s,a_{0}=a] + \gamma E_{p^{\pi}}[\sum_{1\leq t\leq T}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
           &= r(s,a) + \gamma E_{p^{\pi}(a_{1},s_{1},...,a_{T},s_{T}|s_{0}=s,a_{0}=a)}[\sum_{1 \leq t\leq T}\gamma^{t-1}r(s_{t},a_{t})]
\end{align}

But:

\begin{align}
p^{\pi}(a_{1},s_{1},...,a_{T},s_{T}|s_{0}=s,a_{0}=a) &= p^{\pi}(a_{1}, s_{1}|s_0 = s, a_0 = a) \times p^{\pi}(a_{2},s_{2},...,a_{T},s_{T}|s_{0},a_{0}, a_{1}, s_{1}) \\
           &= p^{\pi}(a_{1}, s_{1}|s_0 = s, a_0 = a) \times p^{\pi}(a_{2},s_{2},...,a_{T},s_{T}|a_{1}, s_{1})
\end{align}

because of the Markov property of $\{(a_{t},s_{t})_{t\leq T}\}$

So:

\begin{align}
Q^\pi(s,a) &= r(s,a) + \gamma E_{p^{\pi}(a_{1},s_{1},...,a_{T},s_{T}|s_{0}=s,a_{0}=a)}[\sum_{1 \leq t\leq T}\gamma^{t-1}r(s_{t},a_{t})] \\
           &= r(s,a) + \gamma E_{p^{\pi}(a_{1}, s_{1}|s_0 = s, a_0 = a)}[E_{p^{\pi}(a_{2},s_{2},...,a_{T},s_{T}|a_{1}, s_{1})}[\sum_{1 \leq t\leq T}\gamma^{t-1}r(s_{t},a_{t})]]\\
           &= r(s,a) + \gamma E_{p^{\pi}(a_{1}, s_{1}|s_0 = s, a_0 = a)}[E_{p^{\pi}(a_{2},s_{2},...,a_{T},s_{T})}[\sum_{1 \leq t\leq T}\gamma^{t-1}r(s_{t},a_{t})|a_{1}, s_{1}]] \\
           &= r(s,a) + \gamma E_{p^{\pi}(a_{1}, s_{1}|s_0 = s, a_0 = a)}[Q^\pi(s_1,a_1)] \\
           &= E_{p^{\pi}(a_{1}, s_{1}|s_0 = s, a_0 = a)}[r(s,a) + \gamma Q^\pi(s_1,a_1)] \\
           &= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align}

I.e : \begin{equation*}\boxed{
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]}
\end{equation*}


**2)** Then, we have:

\begin{align}
Q^*(s,a) &= \max_{\pi} Q^\pi(s,a)\\
           &= \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]\\
           &= r(s,a) +  \gamma \max_{\pi}E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')]
\end{align}

But:

\begin{align}
\max_{\pi}E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')] &= \max_{\pi} \sum\limits_{s', a'} p(s'|s,a)\pi(a'|s) Q^\pi(s',a')\\
           &= \max_{\pi} \sum\limits_{s'} p(s'|s,a) \sum\limits_{a'} \pi(a'|s) Q^\pi(s',a')\\
           &= \sum\limits_{s'} p(s'|s,a) \max_{\pi}\sum\limits_{a'} \pi(a'|s) Q^\pi(s',a')
\end{align}

So, we want to maximize $\sum\limits_{a'} \pi(a'|s) Q^\pi(s',a')$ by choosing the policy $\pi$. 

But we know that $\forall s,a \:Q^\pi(s,a) \leq Q^*(s,a)$, so: by choosing $\pi_0 = \pi^* \: \forall s \neq s'$ and $ \: \pi_0(a'|s') =1 $ for $a' = \underset{a}{\mathrm{argmax}} Q^*(s',a)$, we have:

$\sum\limits_{a'} \pi(a'|s) Q^\pi(s',a') \leq \sum\limits_{a'} \pi(a'|s) Q^*(s',a') \leq  Q^*(s',\underset{a}{\mathrm{argmax}} Q^*(s',a)) = \sum\limits_{a'} \pi_0(a'|s) Q^{\pi_0}(s',a')$. We found the policy that maximizes.

So:

\begin{align}
\max_{\pi}E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')] &= \sum\limits_{s'} p(s'|s,a) \max_{\pi}\sum\limits_{a'} \pi(a'|s) Q^\pi(s',a')\\
           &= \sum\limits_{s'} p(s'|s,a) \max_{a'} Q^*(s',a') \\
           &= E_{s' \sim p(.|s,a)}[\max_{a'} Q^*(s',a')]
\end{align}

Then, we conclude that:

\begin{align}
Q^*(s,a) &= \max_{\pi} Q^\pi(s,a)\\
           &= \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]\\
           &= r(s,a) +  \gamma \max_{\pi}E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')] \\
           &= r(s,a) +  \gamma E_{s' \sim p(.|s,a)}[\max_{a'} Q^*(s',a')] \\
           &= \boxed{E_{s' \sim p(.|s,a)}[r(s,a) + \gamma \max_{a'} Q^*(s',a')]}
\end{align}

**3)**

Finally:


We want to find a policy (a Q function) such that : $Q^\pi(s,a) = E_{s' \sim p(.|s,a)}[r(s,a) + \gamma \max_{a'} Q^\pi(s',a')]$ 

i.e $Q^\pi(s,a) = r(s,a) + \gamma \max_{a'} Q^\pi(s',a')$ in expectancy w.r.t $s'$. 

So, we want $Q^\pi(s,a) - r(s,a) + \gamma \max_{a'} Q^\pi(s',a')$ close to 0 in expectancy w.r.t $s'$.

So, we deduce a plausible objective:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) > self.max_memory:
            del self.memory[0]
        self.memory.append(m)

    def random_access(self):
        random_index = np.random.randint(0,len(self.memory))
        return self.memory[random_index]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
from keras.models import Model, Sequential
from keras.layers import Dropout, Input, Conv1D, MaxPooling1D, Flatten, Dense

class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
        
        # Q learned by the agent (init with zero matrix)
        self.model = Sequential()

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.array([s])))
        
        
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            
            prev_state, state,  action, reward, game_over_ = self.memory.random_access()
            
            target_q[i, :] = self.model.predict(np.array([state]))

            if game_over_:
                y = reward
            else:
                y = reward + self.discount*np.max(self.model.predict(np.array([state])))
                
            target_q[i, action]= y # change target for this action
            input_states[i] = prev_state
            
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(np.array(input_states), np.array(target_q))


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

        
        
            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Dense(100, input_shape=(5,5,self.n_state), activation='relu'))
        model.add(Dense(100, activation='relu'))
        model.add(Flatten())
        model.add(Dense(4, activation='relu'))


        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model        

**Test of FC**

In [12]:
from keras import backend as K 
K.clear_session()
epochs_train = 10
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/010 | Loss 0.0030 | Win/lose count 4.0/4.0 (0.0)
Epoch 001/010 | Loss 0.0046 | Win/lose count 2.0/3.0 (-1.0)
Epoch 002/010 | Loss 0.0051 | Win/lose count 5.5/6.0 (-0.5)
Epoch 003/010 | Loss 0.0122 | Win/lose count 0.5/0 (0.5)
Epoch 004/010 | Loss 0.0023 | Win/lose count 4.0/1.0 (3.0)
Epoch 005/010 | Loss 0.0118 | Win/lose count 2.5/1.0 (1.5)
Epoch 006/010 | Loss 0.0073 | Win/lose count 2.5/6.0 (-3.5)
Epoch 007/010 | Loss 0.0115 | Win/lose count 1.0/3.0 (-2.0)
Epoch 008/010 | Loss 0.0268 | Win/lose count 5.0/5.0 (0.0)
Epoch 009/010 | Loss 0.0061 | Win/lose count 3.5/3.0 (0.5)


**Stopping criterium:**

If we want a stopping criterion for the learning. A criterion could be to **check if the policy is updated or not**. However, it's quite expensive because the number of state is big. It's the number of combination of cheese/poison/empty cells in a 5x5 grid (malus 1 : the agent cell): it's $3^{25-1} = 3^{24}$. However, we can decide to reduce the number of state (just for the stopping criterion) by doing something similar to max-pooling for instance. 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [44]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(16, (2, 2), activation='relu', input_shape=(5,5,self.n_state), padding='same'))
        model.add(Conv2D(32, (2, 2), activation='relu', padding='same'))
        model.add(Flatten())
        model.add(Dense(4))
        #model.add(Dropout(0.5))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

**Test of CNN**

In [14]:
K.clear_session()
epochs_train = 10
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/010 | Loss 0.0185 | Win/lose count 3.0/9.0 (-6.0)
Epoch 001/010 | Loss 0.0180 | Win/lose count 1.5/6.0 (-4.5)
Epoch 002/010 | Loss 0.0078 | Win/lose count 2.0/2.0 (0.0)
Epoch 003/010 | Loss 0.0480 | Win/lose count 3.0/4.0 (-1.0)
Epoch 004/010 | Loss 0.0034 | Win/lose count 3.0/7.0 (-4.0)
Epoch 005/010 | Loss 0.0314 | Win/lose count 2.5/5.0 (-2.5)
Epoch 006/010 | Loss 0.0145 | Win/lose count 2.0/7.0 (-5.0)
Epoch 007/010 | Loss 0.0057 | Win/lose count 5.0/3.0 (2.0)
Epoch 008/010 | Loss 0.0034 | Win/lose count 3.0/5.0 (-2.0)
Epoch 009/010 | Loss 0.0138 | Win/lose count 6.0/5.0 (1.0)


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

**Observe the behaviour of the two agents (same temperature = 0.3)**

In [15]:
epochs_test = 11


env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('\n Test of the CNN \n')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('\n Test of the FC \n')
test(agent_fc,env,epochs_test,prefix='fc_test')


 Test of the CNN 

Win/lose count 1.5/4.0. Average score (-2.5)
Win/lose count 2.5/7.0. Average score (-3.5)
Win/lose count 3.0/1.0. Average score (-1.6666666666666667)
Win/lose count 2.5/4.0. Average score (-1.625)
Win/lose count 1.5/4.0. Average score (-1.8)
Win/lose count 3.5/0. Average score (-0.9166666666666666)
Win/lose count 2.0/10.0. Average score (-1.9285714285714286)
Win/lose count 9.0/4.0. Average score (-1.0625)
Win/lose count 0.5/2.0. Average score (-1.1111111111111112)
Win/lose count 1.5/3.0. Average score (-1.15)
Win/lose count 2.0/6.0. Average score (-1.4090909090909092)
Final score: -1.4090909090909092

 Test of the FC 

Win/lose count 2.0/3.0. Average score (-1.0)
Win/lose count 3.5/4.0. Average score (-0.75)
Win/lose count 4.0/2.0. Average score (0.16666666666666666)
Win/lose count 3.5/7.0. Average score (-0.75)
Win/lose count 4.5/10.0. Average score (-1.7)
Win/lose count 2.0/4.0. Average score (-1.75)
Win/lose count 0.5/1.0. Average score (-1.5714285714285714)
Win/

**The agents get mediocre rewards. Let's look at the agents' behaviours with the videos.**

In [16]:
HTML(display_videos('cnn_test10.mp4'))

In [17]:
HTML(display_videos('fc_test10.mp4'))

**Conclusion:** We observe the following issue. The agent tends to oscillates between two empty cells or to stay in small areas of empty cells. In this way the agent avoids poisons. It's a "locally optimal" policy. But the agent also avoids lots of cheese on the board... In a way, the agent is too cautious and do not try to learn a better policy (globally optimal) because to do that the agent need to do more exploration and so to take more risk. The agents gets mediocre total rewards.

**Change the temperature**

In [18]:
epochs_test = 10

for temp in [0.1, 0.4, 0.6, 0.9]:
    
    env = Environment(grid_size=size, max_time=T,temperature=temp)
    agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

    agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

    print(f'\n Test of the CNN, temp = {temp} \n')
    test(agent_cnn,env,epochs_test,prefix=f'{temp}cnn_test')
    print(f'\n Test of the FC, temp = {temp} \n')
    test(agent_fc,env,epochs_test,prefix=f'{temp}fc_test')
    
    


 Test of the CNN, temp = 0.1 

Win/lose count 1.5/1.0. Average score (0.5)
Win/lose count 0.5/1.0. Average score (0.0)
Win/lose count 0/1.0. Average score (-0.3333333333333333)
Win/lose count 1.5/1.0. Average score (-0.125)
Win/lose count 0.5/2.0. Average score (-0.4)
Win/lose count 1.5/1.0. Average score (-0.25)
Win/lose count 0/0. Average score (-0.21428571428571427)
Win/lose count 1.5/1.0. Average score (-0.125)
Win/lose count 1.5/2.0. Average score (-0.16666666666666666)
Win/lose count 0.5/0. Average score (-0.1)
Final score: -0.1

 Test of the FC, temp = 0.1 

Win/lose count 0.5/1.0. Average score (-0.5)
Win/lose count 1.0/5.0. Average score (-2.25)
Win/lose count 0.5/1.0. Average score (-1.6666666666666667)
Win/lose count 0.5/0. Average score (-1.125)
Win/lose count 1.0/6.0. Average score (-1.9)
Win/lose count 1.5/0. Average score (-1.3333333333333333)
Win/lose count 0/0. Average score (-1.1428571428571428)
Win/lose count 1.0/3.0. Average score (-1.25)
Win/lose count 0/0. Averag

**Observe behaviours**

In [19]:
HTML(display_videos('0.1cnn_test9.mp4'))

In [20]:
HTML(display_videos('0.4cnn_test9.mp4'))

In [21]:
HTML(display_videos('0.6cnn_test9.mp4'))

In [22]:
HTML(display_videos('0.9cnn_test9.mp4'))

**Conclusion:**

When the temperature increases, the number of cheeses and poisons increases. Moreover, when the temperature starts to be high (above 0.5), the number of cheese increases and there is just few remaining empty cells to put poison so the number of poison decrease even if the probability in an empty cell to have a poison increases.

What we can say about the **agent's behaviour**. The agent always tries to find an area with empty cells (to stay in). When the temperature is low (there are few cheeses and poisons), the agent will quickly find this kind of area. When the temperature increases there are more cheeses and poisons. The agents should accept to take some risk and go in cheese and maybe poison cells. But when the agent did some "clean up" of the board (remove some cheese/poison), then the agent starts to oscillate in a small group of empty cells that he created.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [42]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    epsilon0 = agent.epsilon
    gamma = 0.9 # parameter to decrease epsilon

    for e in range(epoch):
        
        # At each epoch we decrease epsilon
        agent.set_epsilon(agent.epsilon*gamma)
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [43]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        # new state to foster exploration
        self.malus_position = np.zeros((grid_size,grid_size))
    
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train= False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
            
            
        reward = 0
        if train:
            # We add a malus for already visited positions
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.05

        
        self.t = self.t + 1
        reward += self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        # reset malus_position
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [47]:
K.clear_session()
epochs_train = 20
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.5, epsilon = 0.5, memory_size=2000, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/020 | Loss 0.0227 | Win/lose count 11.5/28.25000000000004 (-16.75000000000004)
Epoch 001/020 | Loss 0.0118 | Win/lose count 16.0/36.75 (-20.75)
Epoch 002/020 | Loss 0.0284 | Win/lose count 12.0/38.94999999999998 (-26.94999999999998)
Epoch 003/020 | Loss 0.0315 | Win/lose count 9.5/56.95000000000001 (-47.45000000000001)
Epoch 004/020 | Loss 0.0350 | Win/lose count 10.5/34.20000000000001 (-23.70000000000001)
Epoch 005/020 | Loss 0.0317 | Win/lose count 11.0/28.950000000000014 (-17.950000000000014)
Epoch 006/020 | Loss 0.0330 | Win/lose count 14.5/40.299999999999976 (-25.799999999999976)
Epoch 007/020 | Loss 0.0505 | Win/lose count 17.5/28.400000000000016 (-10.900000000000016)
Epoch 008/020 | Loss 0.0309 | Win/lose count 10.0/22.150000000000016 (-12.150000000000016)
Epoch 009/020 | Loss 0.0317 | Win/lose count 15.5/19.600000000000012 (-4.100000000000012)
Epoch 010/020 | Loss 0.0473 | Win/lose count 21.0/20.200000000000014 (0.7999999999999865)
Epoch 011/020 | Loss 0.0494 | Win/lo

In [48]:
# Evaluation
epochs_test = 11
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 16.5/4.0. Average score (12.5)
Win/lose count 19.5/4.0. Average score (14.0)
Win/lose count 18.5/10.0. Average score (12.166666666666666)
Win/lose count 22.5/2.0. Average score (14.25)
Win/lose count 23.5/3.0. Average score (15.5)
Win/lose count 25.5/5.0. Average score (16.333333333333332)
Win/lose count 21.0/6.0. Average score (16.142857142857142)
Win/lose count 22.0/3.0. Average score (16.5)
Win/lose count 24.5/5.0. Average score (16.833333333333332)
Win/lose count 19.0/8.0. Average score (16.25)
Win/lose count 24.0/4.0. Average score (16.59090909090909)
Final score: 16.59090909090909


**Conclusion:**

We encourage exploration thanks to the two previous ideas. I added an increasing position malus (+0.05 of malus each time the  agent comes back in a cell)  to avoid oscilations of the agent. I added too a geometrically decreasing epsilon exploration parameter ( with $\gamma = 0.9$) to do less and less exploration.

We can see that the total reward is increased a lot (**total reward about +15**). And the behaviour of the agent seems to be closer to the desired behaviour: the agents explores the board and tries to find the cheeses and to avoid the poisons. The agent is not blocked in a local optimal policy anymore.




***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***